# Week 5 - Plotting

  - [matplotlib documentation](https://matplotlib.org/stable/)
  - [matplotlib examples](https://matplotlib.org/stable/gallery/index.html)

In [ ]:
import numpy as np

x = np.linspace(-np.pi, np.pi, 1000, dtype=float)
y = np.cos(x) + 0.2 * np.cos(10 * x + np.pi / 4)

In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt

plt.close("all")

fig = plt.figure()
ax = fig.subplots(1)
ax.plot(x, y, 'k-')
ax.set_xlim(x[0], x[-1])
ax.set_xlabel('$x$', fontsize='x-large')
ax.set_ylabel('$f(x)$', fontsize='x-large')
ax.axhline(0, color='#888888', linewidth=1)
ax.axvline(0, color='#888888', linewidth=1)

In [ ]:
def f(x, alpha, k, theta):
    return alpha * np.sin(k * x + theta)


fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)

ax[0, 0].plot(x, f(x, 1, 1, 0), 'k-')
ax[0, 0].set_xlabel('$x$')
ax[0, 0].set_ylabel(r'$\sin x$')
ax[0, 0].set_xlim(x[0], x[-1])

ax[0, 1].plot(x, f(x, 1, 1, np.pi / 4), 'k-')
ax[0, 1].set_xlabel('$x$')
ax[0, 1].set_ylabel(r'$\sin \left( x + \frac{\pi}{4} \right)$')
ax[0, 1].set_xlim(x[0], x[-1])

ax[1, 0].plot(x, f(x, 1, 10, 0), 'k-')
ax[1, 0].set_xlabel('$x$')
ax[1, 0].set_ylabel(r'$\sin ( 10 x )$')
ax[1, 0].set_xlim(x[0], x[-1])

ax[1, 1].plot(x, np.exp(-(x + np.pi)) * f(x, 1, 10, 0), 'k-')
ax[1, 1].set_xlabel('$x$')
ax[1, 1].set_ylabel(r'$\exp^{-x + \pi} \sin ( 10 x )$')
ax[1, 1].set_xlim(x[0], x[-1])

# Worked example: Mandelbrot set

  - [Wikipedia: Mandelbrot set](https://en.wikipedia.org/wiki/Mandelbrot_set)

For any given complex number $c \in \mathbb{C}$ we can define the sequence

$$z_0 = 0,$$
$$z_{n + 1} = z_n^2 + c \quad \text{for} ~ n \ge 0.$$

The Mandelbrot set is the set of complex $c$ such that all elements in the sequence have bounded magnitude, which is equivalent to all elements in the sequence having mangitude less than or equal to $2$,

$$M= \left\{ c \in \mathbb{C} : | z_n | \le 2 \quad \forall n \right\}.$$

In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as np

plt.close("all")


def escape_iterations(c, max_its):
    z = 0
    for n in range(max_its):
        if abs(z) > 2:
            return n
        z = z ** 2 + c
    return max_its


def escape_iterations_vectorized(c, max_its):
    N = np.full_like(c, max_its, dtype=int)
    z = np.zeros_like(c, dtype=complex)
    for n in range(max_its):
        N = np.where(np.logical_and(N == max_its, abs(z) > 2),
                     n, N)
        z = np.where(N == max_its, z ** 2 + c, z)
    return N


N_x, N_y, max_its = 256, 256, 64
c_r_min, c_r_max = -2, 2
c_i_min, c_i_max = -2, 2
c_r = np.linspace(c_r_min, c_r_max, N_x)
c_i = np.linspace(c_i_min, c_i_max, N_y)


def plot(c_r, c_i, max_its, fig=None):
    C_r, C_i = np.meshgrid(c_r, c_i, indexing='ij')
    C = C_r + 1.j * C_i
    # N = np.zeros_like(C, dtype=int)
    # for i in range(N_x):
    #     for j in range(N_y):
    #         N[i, j] = escape_iterations(C[i, j], max_its=max_its)
    N = escape_iterations_vectorized(C, max_its=max_its)

    if fig is None:
        fig = plt.figure(figsize=(10, 8))
    fig.clf()
    ax = fig.subplots(1)
    m = ax.pcolormesh(C_r, C_i, np.log10(1 + N),
                      cmap='magma_r')
    fig.colorbar(m, ax=ax)
    ax.set_xlim(c_r.min(), c_r.max())
    ax.set_ylim(c_i.min(), c_i.max())
    ax.set_aspect(1)

    return fig, ax


fig, ax = plot(c_r, c_i, max_its)

Redraw the plot with the current axis limits.

In [ ]:
c_r_min, c_r_max = ax.get_xlim()
c_i_min, c_i_max = ax.get_ylim()
c_r = np.linspace(c_r_min, c_r_max, N_x)
c_i = np.linspace(c_i_min, c_i_max, N_y)

fig, ax = plot(c_r, c_i, max_its, fig=fig)

Interactive version (beyond the scope of this course!).

In [ ]:
def button_release_event_handler(event):
    global c_r_min, c_r_max
    global c_i_min, c_i_max
    global fig, ax

    if ax.get_xlim() != (c_r_min, c_r_max) \
            or ax.get_ylim() != (c_i_min, c_i_max):
        c_r_min, c_r_max = ax.get_xlim()
        c_i_min, c_i_max = ax.get_ylim()
        c_r = np.linspace(c_r_min, c_r_max, N_x)
        c_i = np.linspace(c_i_min, c_i_max, N_y)

        fig, ax = plot(c_r, c_i, max_its, fig=fig)


fig.canvas.mpl_connect('button_release_event', button_release_event_handler)